# VoxelNet: End-to-End Learning for Point Cloud Based 3D Object Detection

## Paper Reviews

### Prior Reseach

기존의 3D object detection은 Point Cloud data를 Region proposal network(Faster R-CNN)을 이용하기 위해 hand-crafted feature로 만드는 과정이 필요했음  
하지만 **PointNet**을 비롯한 Point Cloud로의 end-to-end 3D representation learning은 결과적으로 성공적이었음  
→ 2D에서 Faster R-CNN이 그랬듯이 3D object detection 또한 3D feature extraction과 RPN이 **end-to-end**로 학습될 수 있어야 함!

∴ 3D point cloud input으로 **end-to-end 2-stage object detection**을 수행할 수 있는 **VoxelNet** 를 제안

<p align="center">
<img width="865" alt="1" src="https://user-images.githubusercontent.com/86907286/200104865-d9d60bd9-1c65-4d77-9bac-229c07ffe34f.png">
</p>



### Voxel Feature Learning Network

#### Voxel Partition

RPN structure를 활용하기 위해 LiDAR를 통해서 얻어지는 Point Cloud data는 **Voxel**의 형태로 변환되어야 함  
기존에는 이를 hand-crafted하게 얻었지만 이를 Point Cloud를 통한 feature learning으로 학습할 수 있는 구조로 변환되어야 함    
→ 3D space 상의 Point Cloud $(z, x, y)$가 존재하는 closed domain $[D, H, W]$를 **적절한 크기의 partition size $ v_D, v_H, v_W $로 split하여 Voxel로 간주할 필요가 있음**  

∴ 이렇게 얻은 하나의 group을 통해 하나의 Voxel에서의 representation으로 보고 feature learning을 진행해야 함  
→ Point Cloud는 기본적으로 **non-uniform**하게 분포하게 되기 때문에 Voxel로 표현된 group 내부의 point 수는 일정하지 않게 됨

#### Random Sampling

RiDAR를 통해 얻어진 Point Cloud Data는 거의 ~100k에 달하는 수의 point로 구성됨  
이는 non-uniform한 분포를 갖는다는 점과 함께 각 Voxel마다 point의 편차가 큰 동시에 모두 사용하기에는 연산량이 너무 많아진다는 문제로 이어짐  

∴ 하나의 Voxel이 최대 $T$ points만 갖게 할 수 있도록 만들어 편차를 줄이고 연산량을 절약할 수 있음

#### Stacked Voxel Feature Encoding

앞선 과정을 통해 얻어진 group을 통해서 실제로 Voxel으로 기능하기 위한 feature learning을 수행해야 할 필요가 있음  
→ 기본적으로 Point Cloud를 통해서 Voxel처럼 표현된 형태이기 때문에 Point Cloud feature extractor의 structure를 가져올 필요가 있음  

**PointNet++** 처럼 하나의 Voxel이 되는 group의 point들의 centroid $v$를 기준으로 그 편차들을 계산  
이를 original point $\mathbf{p}_i = [x_i, y_i, z_i, r_i]^T$에 concatenate하여 $\mathbf{\hat{p}}_i = [x_i, y_i, z_i, r_i, x_i-v_x, y_i-v_y, z_i-v_z] $처럼 재표현  
최종적으로 hierarchical point cloud feature learning structure에 input으로 사용하여 feature learning 진행   
→ shared weight로 Voxel group 내부에서 max-response $\mathbf{\tilde{f}}$를 찾아내고 이를 $\mathbf{p}_i$로 얻어진 feature $\mathbf{f}_i$에 concatenate

∴ single feature layer의 결과로 최대 $T$ point 만큼의 feature vector $\mathbf{f}_{i}^{\text{out}} = [\mathbf{f}_i^T, \mathbf{\tilde{f}}^T]^T$를 얻을 수 있음  
→ 이를 stacking하는 과정을 통해 Point Cloud를 Voxel 내부에서의 context를 의미하는 feature learning으로서 기능하게 만듦

<p align="center">
<img width="421" alt="2" src="https://user-images.githubusercontent.com/86907286/200104868-7f795cf0-ec17-45af-a6ef-4ed1c52812b8.png">
</p>

#### Sparse Tensor Representation


Point Cloud는 non-uniform하기 때문에 대부분의 Voxel group에서는 Point가 존재하지 않는 경우가 발생함  
만약 이를 생각하지 않고 naive하게 computation을 진행하면 **대부분의 Voxel이 연산이 필요 없는 zero-tensor**가 됨  
→ 이는 GPU를 사용하는 시점에서 sparse tensor를 계산하는 행위라 memory efficiency를 저해하게 되므로 dense한 형태로 계산할 수 있는 방법이 필요  

∴ Voxel feature를 계산하는 과정에서 어떤 Voxel이 feature value를 갖고 있는지에 대한 buffer와 해당 feature value를 따로 보관하는 것으로 efficiency 개선 가능

이를 위해 어떤 Voxel의 feature value가 존재하면 feature value는 list에 저장하고 추가적인 **hash table**을 도입  
Voxel의 **coordinate를 key**로 저장하고 feature value **list index를 hash value**로 저장  
→ 이는 lookup이 $\mathcal{O}(1)$, computation을 위해 dense tensor로 변환는 것은 list search 만큼인 $\mathcal{O}(N)$이 요구되므로 효율적

<p align="center">
<img width="402" alt="3" src="https://user-images.githubusercontent.com/86907286/200104869-c9e0fe25-68c5-4102-8d86-33041348744d.png">
</p>


### Convolutional Middle Layer

Voxel Feature Learning Network를 통해 얻어지는 것은 하나의 Voxel grid에서의 Point Cloud data 들의 context 

∴ 전체 Voxel set에서의 representation을 얻기 위해 Voxel 간의 context를 찾는 3D CNN 구조가 추가적으로 필요함  
→ 이를 위해 3D CNN, Batch Normalization, ReLU로 구성된 Network 학습

### Region Proposal Network

실질적인 feature extraction이 완료되면 이를 통해서 Classification, Bounding Box Regression을 수행할 Head가 필요함  
이를 위해 Conv2D, Transposed Conv2D로 구성된 bottleneck Architecture를 사용, Figure에서 표현된 Architecture는 Car Detection   
→ 이 과정에서 4D feature map인 middle layer output을 $Z$ axis에 대해서 펼친 **Bird Eye View**로 reshape하기 때문에 2D Conv가 됨

<p align="center">
<img width="647" alt="4" src="https://user-images.githubusercontent.com/86907286/200104870-52d694f1-900a-4e54-811e-9ee75645d1ff.png">
</p>

### Loss Function

2D space에서의 Bounding Box regression은 center point $(x, y)$, width/height $(h, w)$ 보다 더 많은 regression을 수행해야 함   
추가된 축 $Z$에 대한 좌표, 즉 **높이**인 $z$와 축을 기준으로 **object direction(yaw rotation)** 을 나타내는 $\theta \in [0, 2\pi]$가 더 필요함  
→ 이때 사용하는 anchor가 각 point마다 object $0, \frac{\pi}{2}$ rotation **2개로 이루어져있어** Regression map의 channel이 14로 표현되었음  

이 과정에서 regression에서 ground-truth box에 대한 anchor에서의 residual  $\mathbf{u}^{*}$을 예측할 때 $x, y$에 대해서는 **normalize**가 된 후의 값을 예측하게 만듦  
이는 $x, y$가 anchor box의 diagonal base $d^a = \sqrt{(l^a)^2+(w^a)^2}$에 대해서는 homogenous 하도록 유도하여 **한 axis로 prediction이 튀는 것을 방지**하는 것

$$ \mathbf{u}^{*} = (\Delta x, \Delta y, \Delta z, \Delta l, \Delta w, \Delta h, \Delta \theta) $$

$$ \Delta x = \frac{x^g_c-x^a_c}{d_a}, \Delta y = \frac{y^g_c-y^a_c}{d_a},  \Delta z = \frac{z^g_c-z^a_c}{h_a} $$
$$ \Delta l = \log(\frac{l^g}{l^a}), \Delta w = \log(\frac{w^g}{w^a}), \Delta h = \log(\frac{h^g}{h^a}),  \Delta \theta = \theta^g - \theta^a$$




따라서 2D object detection의 classification Loss와 Regression Loss에서 2개의 parameter가 추가된 형태의 multitask loss가 적용됨  
Classification Loss는 Binary Cross Entropy Loss, Regression Loss는 Smooth-L1 Loss 적용  

$$ \text{smooth}_{L_1}(x) = \begin{cases}
  0.5x^2 & \text{if } |x| < 1, \\
  |x|-0.5 & \text{otherwise}
\end{cases}$$

$$ \mathcal{L} = \alpha \frac{1}{N_{\text{pos}}} \sum_i \mathcal{L}_{\text{cls}}(p_i^{\text{pos}}, 1) + \beta \frac{1}{N_{\text{neg}}} \sum_j \mathcal{L}_{\text{cls}}(p_i^{\text{neg}}, 0) + \frac{1}{N_{\text{pos}}} \sum_i \mathcal{L}_{\text{reg}}(\mathbf{u}_i, \mathbf{u}_i^{*})$$

이때 사용하는 Ground-truth Bounding Box 및 Point Cloud에 perturbation을 통해 Data Augmentation을 진행했음

1. Bounding Box의 rotation $\theta$와 $X, Y, Z$ component에 대해 uniform, gaussian random variable로 perturbation
2. Bounding Box와 그 내부 point에 대한 uniform random variable로의 scale perturbation
3. 모든 Ground Bounding Box와 Point Cloud에 대해서 동일하게 가해지는 $Z$ axis에 대한 rotation perturbation

## Implementation Reviews

VFE layer

In [ ]:
# Voxel Feature Encoding layer
class VFE(nn.Module):

    def __init__(self,cin,cout):
        super(VFE, self).__init__()
        assert cout % 2 == 0
        self.units = cout // 2
        self.fcn = FCN(cin,self.units)

    def forward(self, x, mask):
        # point-wise feauture
        pwf = self.fcn(x)
        #locally aggregated feature
        laf = torch.max(pwf,1)[0].unsqueeze(1).repeat(1,cfg.T,1)
        # point-wise concat feature
        pwcf = torch.cat((pwf,laf),dim=2)
        # apply mask
        mask = mask.unsqueeze(2).repeat(1, 1, self.units * 2)
        pwcf = pwcf * mask.float()

        return pwcf

# Stacked Voxel Feature Encoding
class SVFE(nn.Module):

    def __init__(self):
        super(SVFE, self).__init__()
        self.vfe_1 = VFE(7,32)
        self.vfe_2 = VFE(32,128)
        self.fcn = FCN(128,128)
    def forward(self, x):
        mask = torch.ne(torch.max(x,2)[0], 0)
        x = self.vfe_1(x, mask)
        x = self.vfe_2(x, mask)
        x = self.fcn(x)
        # element-wise max pooling
        x = torch.max(x,1)[0]
        return 

Main structure

In [ ]:
class VoxelNet(nn.Module):
    
    def __init__(self):
        super(VoxelNet, self).__init__()
        self.svfe = SVFE()
        self.cml = CML()
        self.rpn = RPN()

    def voxel_indexing(self, sparse_features, coords):
        dim = sparse_features.shape[-1]

        dense_feature = Variable(torch.zeros(dim, cfg.N, cfg.D, cfg.H, cfg.W).cuda())

        dense_feature[:, coords[:,0], coords[:,1], coords[:,2], coords[:,3]]= sparse_features

        return dense_feature.transpose(0, 1)

    def forward(self, voxel_features, voxel_coords):

        # feature learning network
        vwfs = self.svfe(voxel_features)
        vwfs = self.voxel_indexing(vwfs,voxel_coords)

        # convolutional middle network
        cml_out = self.cml(vwfs)

        # region proposal network

        # merge the depth and feature dim into one, output probability score map and regression map
        psm,rm = self.rpn(cml_out.view(cfg.N,-1,cfg.H, cfg.W))

        return psm, rm

## Reference

https://arxiv.org/abs/1711.06396  
https://github.com/skyhehe123/VoxelNet-pytorch/blob/master/voxelnet.py